In [9]:
import pandas as pd
import talib as ta
import numpy as np

pd.options.display.float_format = '{:.2f}'.format

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
from pathlib import Path
from sys import path

notebook_path = Path.cwd()
SITE = notebook_path.parent
path.append(str(SITE.absolute()))
from libs.c_helpers import *

In [12]:
folder_path = Path(SITE / 'investing/data/exports/indices/')
filenames = [f for f in folder_path.iterdir() if f.is_file()]

cumulative_returns = {}

for filename in filenames:
    ticker = str(filename).split('.')[0].split('/')[-1]
    df = pd.read_csv(filename, infer_datetime_format=True)[:-1]
    
    df['Return'] = 1 + (df['Change'] / 100)
    df['CumReturn'] = df['Return'].cumprod()
    # print(ticker, list(df.tail(1)['CumReturn'])[0])
    cumulative_returns[ticker] = list(df.tail(1)['CumReturn'])[0]

sorted_returns = dict(sorted(cumulative_returns.items(), key=lambda x: x[1], reverse=True))
sorted_returns


{'VNI': 12.850611017078858,
 'HNXMSC': 8.357261916771598,
 'VNI30': 4.135442905827291,
 'HNXCON': 3.4556511797133935,
 'TFTFVAS': 3.15444797086652,
 'HNX30': 3.0364627591219935,
 'VNIMC': 2.9635940364562168,
 'HNXLC': 2.4677486702277407,
 'HNXI': 2.4392258222337198,
 'VNI100': 2.240655826750408,
 'FTFVAS': 2.235965627960124,
 'VNIAS': 2.2160432483478916,
 'VNISC': 2.113614452769996,
 'FTFVTT': 1.1631787701924152,
 'TFTFVTT': 1.1234958300660283,
 'UPCOMI': 0.9171428562344394,
 'TFTFVTTU': 0.8599919541352962}

In [13]:
ticker = "VNI"
filename = SITE / f'investing/data/exports/indices/{ticker}.csv'
df = pd.read_csv(filename, infer_datetime_format=True)[:-1]
df['Volume'] = df['Volume'].astype(int)
df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index)

In [14]:
daily_df = df.copy()
daily_df['Return'] = 1 + (daily_df['Change'] / 100)
daily_df['CumReturn'] = daily_df['Return'].cumprod()
daily_df

,Close,Open,High,Low,Volume,Change,Return,CumReturn
Date,,,,,,,,
2000-07-31,101.55,101.55,101.55,101.55,10,1.55,1.02,1.02
2000-08-02,103.38,103.38,103.38,103.38,0,1.80,1.02,1.03
2000-08-04,105.20,105.20,105.20,105.20,0,1.76,1.02,1.05
2000-08-07,106.92,106.92,106.92,106.92,10,1.63,1.02,1.07
2000-08-09,108.64,108.64,108.64,108.64,20,1.61,1.02,1.09
...,...,...,...,...,...,...,...,...
2024-03-25,1267.86,1281.80,1286.84,1264.51,1240000,-1.09,0.99,12.69
2024-03-26,1282.21,1267.86,1282.50,1264.42,941700,1.13,1.01,12.83
2024-03-27,1283.09,1282.21,1286.72,1276.95,919280,0.07,1.00,12.84


In [15]:
weekly_df = df.resample('W').agg({'Close': 'last', 'Open': 'first', 'High': 'max', 'Low': 'min', 'Volume': 'sum', 'Change': 'sum'})
# weekly_df['AvgVol_1W'] = weekly_df['Volume'].rolling(1).mean()
# weekly_df['AvgVol_2W'] = weekly_df['Volume'].rolling(2).mean()
# weekly_df['AvgVol_5W'] = weekly_df['Volume'].rolling(5).mean()
weekly_df

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2000-08-06,105.20,101.55,105.20,101.55,10,5.11
2000-08-13,110.36,106.92,110.36,106.92,40,4.82
2000-08-20,115.52,112.08,115.52,112.08,40,4.60
2000-08-27,118.84,117.24,118.96,117.24,250,2.86
2000-09-03,113.37,117.00,117.00,113.37,330,-4.68
...,...,...,...,...,...,...
2024-03-03,1258.28,1212.00,1264.39,1210.52,4899290,3.76
2024-03-10,1247.35,1258.28,1277.51,1247.35,5620000,-0.85
2024-03-17,1263.78,1247.35,1276.41,1233.14,4924440,1.34


In [16]:
monthly_df = df.resample('M').agg({'Close': 'last', 'Open': 'first', 'High': 'max', 'Low': 'min', 'Volume': 'sum', 'Change': 'sum'})
monthly_df

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2000-07-31,101.55,101.55,101.55,101.55,10,1.55
2000-08-31,115.15,103.38,118.96,103.38,560,12.71
2000-09-30,120.71,113.37,120.71,112.39,660,4.79
2000-10-31,140.84,120.92,140.84,120.92,690,15.54
2000-11-30,168.73,143.33,168.73,143.33,890,18.23
...,...,...,...,...,...,...
2023-11-30,1094.13,1028.19,1132.75,1020.01,18169420,6.41
2023-12-31,1129.93,1094.13,1135.05,1082.29,15720580,3.25
2024-01-31,1164.31,1129.93,1186.96,1128.32,17351190,3.03


In [17]:
yearly_df = df.resample('Y').agg({'Close': 'last', 'Open': 'first', 'High': 'max', 'Low': 'min', 'Volume': 'sum', 'Change': 'sum'})
yearly_df

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2000-12-31,206.83,101.55,206.83,101.55,3720,73.50
2001-12-31,235.40,210.56,571.04,203.12,19770,21.69
2002-12-31,183.33,231.70,231.70,174.62,37190,-24.12
2003-12-31,166.94,183.41,183.41,130.90,53310,-8.35
2004-12-31,239.29,167.69,280.53,167.69,248260,38.16
2005-12-31,307.50,238.39,325.25,232.41,353140,25.89
2006-12-31,751.77,306.86,815.98,304.23,1120880,94.60
2007-12-31,927.02,751.04,1179.32,741.27,2389680,24.60
2008-12-31,315.62,925.66,926.01,284.06,3405000,-100.82


In [18]:
three_yearly_df = df.resample('3Y').agg({'Close': 'last', 'Open': 'first', 'High': 'max', 'Low': 'min', 'Volume': 'sum', 'Change': 'sum'})
three_yearly_df

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2000-12-31,206.83,101.55,206.83,101.55,3720,73.50
2003-12-31,166.94,210.56,571.04,130.90,110270,-10.78
2006-12-31,751.77,167.69,815.98,167.69,1722280,158.65
2009-12-31,494.77,751.04,1179.32,234.66,16886470,-25.30
2012-12-31,413.73,501.74,551.40,332.28,34292930,-11.44
2015-12-31,579.03,415.50,644.56,413.68,599009870,38.06
2018-12-31,892.54,579.03,1211.34,513.82,130856270,47.16
2021-12-31,1498.28,900.10,1511.68,649.10,325451730,57.30
2024-12-31,1284.09,1508.55,1536.45,873.78,420367460,-10.73
